In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pdx
import numpy as np
from sklearn.model_selection import train_test_split
import joblib


In [8]:
# Load the first dataset
df1 = pd.read_csv('badmintondata.csv')

# Load the second dataset
df2 = pd.read_csv('badmintondata2.csv')

# Combine both datasets
df_combined = pd.concat([df1, df2], ignore_index=True)

# Count rows before cleaning
rows_before_cleaning = df_combined.shape[0]

# Handling missing values
df_combined = df_combined.dropna()  # Drop rows with missing values

# Handling outliers (Using z-score)
z_scores_combined = (df_combined - df_combined.mean()) / df_combined.std()  # Calculate z-scores
threshold = 3  # Set a threshold for outliers
outliers_combined = (z_scores_combined.abs() > threshold).any(axis=1)  # Find rows with outliers
df_combined_cleaned = df_combined[~outliers_combined].copy()  # Filter out rows with outliers

# Identify consecutive rows of all zeros
mask = (df_combined == 0).all(axis=1)
groups = mask.cumsum()

# Filter out consecutive rows of all zeros
df_combined_cleaned = df_combined[~mask]
df_combined_cleaned.to_csv('cleaned_dataset.csv', index = False)

# Count rows after cleaning
rows_after_cleaning = df_combined_cleaned.shape[0]

print("Number of rows before cleaning:", rows_before_cleaning)
print("Number of rows after cleaning:", rows_after_cleaning)


Number of rows before cleaning: 315200
Number of rows after cleaning: 168842


In [6]:
# Generate scatterplot for the cleaned combined dataset
fig, axes = plt.subplots(1, 2, figsize=(15, 8), subplot_kw={'projection': '3d'})

# Cleaned Combined Dataset
axes[0].scatter(df_combined_cleaned['SHUTTLECOCK POSITIION IN AIR(X) metres'], df_combined_cleaned['SHUTTLECOCK POSITIION IN AIR(Y) metres'], df_combined_cleaned['SHUTTLECOCK POSITIION IN AIR(Z) metres'], df_combined_cleaned['SHUTTELCOCK SLANT ANGLE TO SIDELINE(DEGREE)'], s=5)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_zlabel('Z')
axes[0].set_title('Cleaned Combined Dataset')
axes[0].grid(True)

# Uncleaned Combined Dataset
axes[1].scatter(df_combined['SHUTTLECOCK POSITIION IN AIR(X) metres'], df_combined['SHUTTLECOCK POSITIION IN AIR(Y) metres'], df_combined['SHUTTLECOCK POSITIION IN AIR(Z) metres'], s=5)
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].set_zlabel('Z')
axes[1].set_title('Uncleaned Combined Dataset')
axes[1].grid(True)

plt.tight_layout()
plt.show()


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Error in callback <function _draw_all_if_interactive at 0x134837880> (for post_execute):


AttributeError: 'Path3DCollection' object has no attribute '_offsets3d'

AttributeError: 'Path3DCollection' object has no attribute '_offsets3d'

<Figure size 1500x800 with 2 Axes>

In [9]:
# Group the data before each group of zeros
grouped_df = df_combined_cleaned.groupby(groups, group_keys=False)

# Function to add a Time field and SERVE_ID to a group
def add_time_and_serve_id(group):
    group['TIME'] = np.arange(len(group)) * 10  # Multiply by 10 to get time in ms
    group['SERVE_ID'] = group.name  # Assign the group name as the SERVE_ID

    # Reset time to 0 at the start of each group
    group.loc[group.index[0], 'TIME'] = 0
    
    return group

# Apply a TIME-based transformation and add SERVE_ID to each group
grouped_df = grouped_df.apply(add_time_and_serve_id)

# Extract unique groups
serve_ids = grouped_df['SERVE_ID'].unique()

# Split the groups into testing set and sample set (split into a testing set and a sample set using a 70:30 ratio)
train_groups, test_groups = train_test_split(serve_ids, test_size=0.3, random_state=42)

# Create the training set with highest point labels
train_data = []
for group in train_groups:
    group_data = grouped_df[grouped_df['SERVE_ID'] == group].copy()
    highest_point = group_data['SHUTTLECOCK POSITIION IN AIR(Z) metres'].max()
    group_data['HIGHEST_POINT'] = highest_point
    train_data.append(group_data)

train_df = pd.concat(train_data)

# Create the testing set with highest point labels
test_data = []
for group in test_groups:
    group_data = grouped_df[grouped_df['SERVE_ID'] == group].copy()
    highest_point = group_data['SHUTTLECOCK POSITIION IN AIR(Z) metres'].max()
    group_data['HIGHEST_POINT'] = highest_point
    test_data.append(group_data)

test_df = pd.concat(test_data)

# Print two groups from train_groups
print("Two groups from train_groups:")
for group in train_groups[:2]:
    print(f"Group: {group}")
    print(train_df[train_df['SERVE_ID'] == group])
    print("\n")

# Print two groups from test_groups
print("Two groups from test_groups:")
for group in test_groups[:2]:
    print(f"Group: {group}")
    print(test_df[test_df['SERVE_ID'] == group])
    print("\n")

Two groups from train_groups:
Group: 63153
        HUMAN PLAYER POSITION (X) metres  HUMAN PLAYER POSITION (Y) metres  \
155400                                 4                                 1   
155401                                 4                                 1   
155402                                 4                                 1   
155403                                 4                                 1   
155404                                 4                                 1   
...                                  ...                               ...   
155548                                 4                                 1   
155549                                 4                                 1   
155550                                 4                                 1   
155551                                 4                                 1   
155552                                 4                                 1   

        INITITAL VEL

In [10]:
import random

# Combine fields into a single dataframe
combined_group = pd.concat([train_df, test_df])

# Reset the index of the combined group
combined_group.reset_index(drop=True, inplace=True)

# Select two random groups from the combined group dataframe
random_groups = random.sample(list(combined_group['SERVE_ID'].unique()), 2)

# Print data for the randomly selected groups
for group_id in random_groups:
    group_data = combined_group.loc[combined_group['SERVE_ID'] == group_id]
    print(f"Group {group_id}:")
    print(group_data)
    print()


Group 114678:
        HUMAN PLAYER POSITION (X) metres  HUMAN PLAYER POSITION (Y) metres  \
154037                                 4                                 3   
154038                                 4                                 3   
154039                                 4                                 3   
154040                                 4                                 3   
154041                                 4                                 3   
154042                                 4                                 3   
154043                                 4                                 3   
154044                                 4                                 3   
154045                                 4                                 3   
154046                                 4                                 3   
154047                                 4                                 3   
154048                                 4          

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
import joblib
from scipy.interpolate import interp1d

# Separate the features and target variable in the training set
X_train = train_df[["SHUTTLECOCK POSITIION IN AIR(X) metres", "SHUTTLECOCK POSITIION IN AIR(Y) metres", "SHUTTLECOCK POSITIION IN AIR(Z) metres"]]
y_train = train_df['HIGHEST_POINT']

print("Training the model...")
# Train the model
maxDepthArr = [1,3,5,7,9,11,15,30,50,100]
rmseErrorArr = []
rSqrErrorArr = []
maeErrorArr = []
cvRmseErrorArr = []


for idx, i in enumerate(maxDepthArr):
    model = RandomForestRegressor(max_depth=i)
    model.fit(X_train, y_train)
    print("Model training complete.")

    # Separate the features and target variable in the testing set
    X_test = test_df[["SHUTTLECOCK POSITIION IN AIR(X) metres", "SHUTTLECOCK POSITIION IN AIR(Y) metres", "SHUTTLECOCK POSITIION IN AIR(Z) metres"]]
    y_test = test_df['HIGHEST_POINT']

    print("Making predictions on the testing set...")
    # Make predictions on the testing set
    y_pred = model.predict(X_test)
    print("Prediction complete.")

    # Calculate performance indicators
    r2 = model.score(X_test, y_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    cv_rmse = -np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error'))
    
    rSqrErrorArr.append(r2)
    rmseErrorArr.append(rmse)
    maeErrorArr.append(mae)
    cvRmseErrorArr.append(cv_rmse)
    
    

    print("\nPerformance Indicators for maxDepth =", i)
    print(f"R^2: {r2}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"CV RMSE: {cv_rmse}")
    

# plot depth vs error graph
# Plotting the Graph
x=maxDepthArr

plt.plot(x, rSqrErrorArr)
plt.xlabel("Tree Depth")
plt.ylabel("r^2 Score")
plt.legend()
plt.title('Tree Depth vs r^2 Score')
plt.show()


plt.plot(x, rmseErrorArr)
plt.xlabel("Tree Depth")
plt.ylabel("rmse Error")
plt.legend()
plt.title('Tree Depth vs rmse Error')
plt.show()

# Plotting the Graph

plt.plot(x, maeErrorArr)
plt.xlabel("Tree Depth")
plt.ylabel("mae Error")
plt.legend()
plt.title('Tree Depth vs mae Error')
plt.show()

# Plotting the Graph

plt.plot(x, cvRmseErrorArr)
plt.xlabel("Tree Depth")
plt.ylabel("CV rmse Error")
plt.legend()
plt.title('Tree Depth vs CV rsme Error')
plt.show()




# Export the model
joblib.dump(model, 'regression_model.joblib')
print("Model saved as regression_model.joblib.")


Training the model...
Model training complete.
Making predictions on the testing set...
Prediction complete.

Performance Indicators for maxDepth = {}: 1
R^2: 0.31949309039921103
RMSE: 1.02296670736541
MAE: 0.8037365580609585
CV RMSE: 1.0066760784941655
Model training complete.
Making predictions on the testing set...
Prediction complete.
